In [1]:
import warnings
warnings.filterwarnings('ignore')

# 교차 검증과 그리드 서치
- 머신러닝을 사용할때 모델의 정확도를 측정하기 위해 반드시 사용해야하는 방법
- 딥러닝시에는 데이터의 크기가 크므로 이 방법은 사용할 필요가 없다.

In [2]:
import pandas as pd
wine = pd.read_csv("../Data/wine.csv")
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
# Feature, Target
data = wine[['alcohol','sugar','pH']].to_numpy()
target= wine['class'].to_numpy()

# 검증 세트 추가

In [12]:
# 전체 세트중 훈련세트와 테스트 세트를  8 : 2의 기준으로 분라한다.
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split (
    data, target, test_size=0.2,random_state= 42
)

In [13]:
# 훈련세트 중 훈련세트와 검증세트를 8:2의 기준으로 분리한다.
sub_input, val_input, sub_traget, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [14]:
# 훈련세트, 검증세트, 테스트의 크기 구하기
print('훈련 세트 :',sub_input.shape)
print('검증 세트:',val_input.shape)
print('테스트 세트',test_input.shape)

훈련 세트 : (4157, 3)
검증 세트: (1040, 3)
테스트 세트 (1300, 3)


In [16]:
# 훈련세트와 검증세트를 결정트리로 모델 만들기

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_traget)

print('Train score:', dt.score(sub_input,sub_traget))
print('vaild score:', dt.score(val_input,val_target))


Train score: 0.9971133028626413
vaild score: 0.8586538461538461


--- 
# 교차검증

In [17]:
from sklearn.model_selection import cross_validate
scores =cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00617409, 0.00597095, 0.00970292, 0.006217  , 0.00603628]),
 'score_time': array([0.001122  , 0.0006299 , 0.0013299 , 0.00065804, 0.00077486]),
 'test_score': array([0.86730769, 0.85      , 0.88065448, 0.85081809, 0.84023099])}

In [19]:
import numpy as np
np.mean(scores['test_score'])

0.8578022506848301

---
# KFold : 분할기를 사용한교차 검증

In [20]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=5) #  기본 splits는 5
scores = cross_validate(dt, train_input, train_target, cv=splitter)
scores

{'fit_time': array([0.0064168 , 0.00590587, 0.00644088, 0.00563812, 0.00630307]),
 'score_time': array([0.00082827, 0.00072408, 0.00060225, 0.00082898, 0.00074172]),
 'test_score': array([0.87307692, 0.85192308, 0.87680462, 0.85466795, 0.8373436 ])}

In [21]:
np.mean(scores['test_score'])

0.8587632338787297

In [22]:
# KFold의 Fold를 10개 나누어서 교차검증
splitter = StratifiedKFold(n_splits=10, shuffle= True, random_state=42) #  기본 splits는 5
scores = cross_validate(dt, train_input, train_target, cv=splitter)
scores
# np.mean(scores['test_score'])

{'fit_time': array([0.00761771, 0.00715399, 0.0071969 , 0.00665212, 0.0069859 ,
        0.00714111, 0.00727391, 0.00669718, 0.00651193, 0.0061748 ]),
 'score_time': array([0.00052929, 0.00058603, 0.00051594, 0.00051975, 0.00055408,
        0.00050306, 0.00052094, 0.00047493, 0.00051308, 0.00054908]),
 'test_score': array([0.83653846, 0.88269231, 0.84423077, 0.85192308, 0.85384615,
        0.86730769, 0.86153846, 0.85549133, 0.84971098, 0.87475915])}

In [23]:
splitter = StratifiedKFold(n_splits=10) #  기본 splits는 5
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8637587075737365